<a href="https://colab.research.google.com/github/maleehahassan/HIDA_Into_to_DL/blob/main/09_simple_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Deep Learning with PyTorch

This notebook implements a basic deep learning model using PyTorch to classify MNIST digits. We'll cover:
- Building a simple neural network
- Training and validation process
- Performance visualization

## Step 1: Import Required Libraries

We need the following libraries:
- torch: Main PyTorch library
- torch.nn: Neural network modules
- torch.optim: Optimization algorithms
- torchvision: For accessing the MNIST dataset
- matplotlib: For visualization

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

## Step 2: Set Random Seed

Setting a random seed ensures reproducible results across different runs.

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)

## Step 3: Define Neural Network Architecture

Create a simple neural network with:
1. Input layer: 784 neurons (28x28 MNIST images flattened)
2. Hidden layer: 128 neurons with ReLU activation
3. Output layer: 10 neurons (one for each digit)

The network includes:
- Flattening operation to convert 2D images to 1D
- Fully connected layers
- ReLU activation for non-linearity

In [ ]:
# Define the neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Input layer (28x28 = 784 pixels) -> Hidden layer (128 neurons) -> Output layer (10 classes)
        self.flatten = nn.Flatten()  # Flatten the 28x28 image to 784 pixels
        self.layers = nn.Sequential(
            nn.Linear(784, 128),     # First layer: 784 -> 128
            nn.ReLU(),               # Activation function
            nn.Linear(128, 10)       # Output layer: 128 -> 10 (number of classes)
        )

    def forward(self, x):
        x = self.flatten(x)          # Forward propagation
        return self.layers(x)

## Step 4: Define Training Function

The `train_model` function handles:
1. Training phase:
   - Forward propagation
   - Loss computation
   - Backward propagation
   - Weight updates
2. Validation phase:
   - Model evaluation
   - Metrics calculation

It tracks:
- Loss values
- Accuracy metrics
- Training progress

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        # Training phase
        model.train()           # puts model in training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()     # Clear gradients
            loss.backward()          # Compute gradients
            optimizer.step()         # Update weights

            # Calculate training statistics
            running_loss += loss.item()                     # add up loss
            _, predicted = torch.max(outputs.data, 1)       # get predicted classes
            total += labels.size(0)                         # count how many images seen
            correct += (predicted == labels).sum().item()   # count correct ones

        # Calculate training metrics
        train_loss = running_loss / len(train_loader)           # Average loss over all training batches
        train_accuracy = 100 * correct / total                  # Accuracy (%) = (number of correct predictions / total samples) * 100

        # Save the results so we can plot them later
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Validation phase
        model.eval()     # puts model in evaluation mode
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # No need to compute gradients during validation
            for images, labels in val_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Calculate validation metrics
        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * correct / total
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}]:')
        print(f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
        print('-' * 60)

    return train_losses, val_losses, train_accuracies, val_accuracies

## Step 5: Define Visualization Function

The `plot_metrics` function creates two plots:
1. Training and Validation Loss
2. Training and Validation Accuracy

This helps visualize:
- Model convergence
- Potential overfitting
- Learning progress

In [ ]:
def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs = range(1, len(train_losses) + 1)

    # Create a figure with two subplots
    plt.figure(figsize=(12, 5))

    # Plot losses
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'b-', label='Training Loss')
    plt.plot(epochs, val_losses, 'r-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot accuracies
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, 'b-', label='Training Accuracy')
    plt.plot(epochs, val_accuracies, 'r-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

## Step 6: Main Execution Function

The `main` function orchestrates the entire process:

1. Hyperparameter setup:
   - Batch size: 64
   - Learning rate: 0.001
   - Number of epochs: 10

2. Data preparation:
   - Load MNIST dataset
   - Apply transformations
   - Split into train/validation sets

3. Model setup:
   - Initialize neural network
   - Define loss function
   - Configure optimizer

4. Training and visualization:
   - Train the model
   - Plot performance metrics

In [ ]:
def main():
    # Hyperparameters
    batch_size = 64
    learning_rate = 0.001
    num_epochs = 10

    # Data preprocessing
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # MNIST mean and std
    ])

    # Load MNIST dataset
    full_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)

    # Split into training and validation sets (80% train, 20% validation)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize the model, loss function, and optimizer
    model = SimpleNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model and get metrics
    train_losses, val_losses, train_accuracies, val_accuracies = train_model(
        model, train_loader, val_loader, criterion, optimizer, num_epochs
    )

    # Plot the results
    plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies)

## Step 7: Execute Training

Run the main function when the script is executed directly.
This is a Python idiom that ensures the training only runs when intended.

In [ ]:
if __name__ == '__main__':
    main()